In [10]:
import pandas as pd
import numpy as np
import requests
from pandas.io.json import json_normalize
import datetime

## Step 1: Get Data into Pandas Dataframe<br>
## Step 2: Drop useless columns<br>
## Step 3: Make Dataset only where asset_type == "Stock"<br>
## Step 4: Make new column of Ashley's method. Fill in Price at date of transaction<br>
## Step 5: Save Date to csv, move on to Process2.ipynb<br>
## ...<br>
## <br>
## End product:<br>
# owner	ticker	asset_type	type	amount	senator purchase_date   purchase_price  sell_date(Hard)   sell_price<br>

## Step 1: Get Data into PD

In [11]:
senate_url = "https://senate-stock-watcher-data.s3-us-west-2.amazonaws.com/aggregate/all_transactions.json"
#house_url = "https://house-stock-watcher-data.s3-us-west-2.amazonaws.com/data/all_transactions.json"
senate_data = pd.read_json(senate_url)
#house_data = pd.read_json(house_url)
senate_data.head()

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date
0,09/22/2021,Spouse,--,"Federal Farm Credit Banks Bond <div class=""tex...",Corporate Bond,Purchase,"$500,001 - $1,000,000",--,Rick Scott,https://efdsearch.senate.gov/search/view/ptr/d...,10/28/2021
1,09/22/2021,Spouse,--,"Federal Farm Credit Banks Bond <div class=""tex...",Corporate Bond,Purchase,"$250,001 - $500,000",--,Rick Scott,https://efdsearch.senate.gov/search/view/ptr/d...,10/28/2021
2,10/21/2021,Spouse,--,Memphis Tenn Storm Water Sys Revenue Bond <div...,Municipal Security,Purchase,"$500,001 - $1,000,000",--,Rick Scott,https://efdsearch.senate.gov/search/view/ptr/d...,10/28/2021
3,10/21/2021,Spouse,--,Memphis Tenn Storm Water Sys Revenue Bond <div...,Municipal Security,Purchase,"$250,001 - $500,000",--,Rick Scott,https://efdsearch.senate.gov/search/view/ptr/d...,10/28/2021
4,10/21/2021,Spouse,--,Memphis Tenn Storm Water Sys Revenue Bond <div...,Municipal Security,Purchase,"$250,001 - $500,000",--,Rick Scott,https://efdsearch.senate.gov/search/view/ptr/d...,10/28/2021


## Step 2: Drop Unecessary columns

In [12]:
senate_data.drop(columns = ['ptr_link','asset_description','comment','disclosure_date'], inplace = True)
senate_data.head()

,transaction_date,owner,ticker,asset_type,type,amount,senator
0,09/22/2021,Spouse,--,Corporate Bond,Purchase,"$500,001 - $1,000,000",Rick Scott
1,09/22/2021,Spouse,--,Corporate Bond,Purchase,"$250,001 - $500,000",Rick Scott
2,10/21/2021,Spouse,--,Municipal Security,Purchase,"$500,001 - $1,000,000",Rick Scott
3,10/21/2021,Spouse,--,Municipal Security,Purchase,"$250,001 - $500,000",Rick Scott
4,10/21/2021,Spouse,--,Municipal Security,Purchase,"$250,001 - $500,000",Rick Scott


## Step 3: Save only asset_type == "Stock"

In [13]:
new_data = senate_data[senate_data['asset_type'] == "Stock"]
no_ticker = new_data[new_data['ticker'] == '--']
new_data = new_data.drop(no_ticker.index)
print(new_data.shape, new_data['asset_type'].unique())

(5789, 7) ['Stock']


In [14]:
new_data.head()

,transaction_date,owner,ticker,asset_type,type,amount,senator
5,09/30/2021,Joint,PG,Stock,Purchase,"$1,001 - $15,000",Thomas H Tuberville
6,09/30/2021,Joint,NFLX,Stock,Purchase,"$1,001 - $15,000",Thomas H Tuberville
7,09/30/2021,Joint,MPWR,Stock,Purchase,"$1,001 - $15,000",Thomas H Tuberville
8,09/30/2021,Joint,KEYS,Stock,Purchase,"$1,001 - $15,000",Thomas H Tuberville
9,09/30/2021,Joint,KLAC,Stock,Purchase,"$1,001 - $15,000",Thomas H Tuberville


## Step 4: Ashley's Method

In [19]:
new_data['basis'] =0
new_data['SPY basis'] =0
new_data.head()

,transaction_date,owner,ticker,asset_type,type,amount,senator,basis,SPY basis
5,09/30/2021,Joint,PG,Stock,Purchase,"$1,001 - $15,000",Thomas H Tuberville,0,0
6,09/30/2021,Joint,NFLX,Stock,Purchase,"$1,001 - $15,000",Thomas H Tuberville,0,0
7,09/30/2021,Joint,MPWR,Stock,Purchase,"$1,001 - $15,000",Thomas H Tuberville,0,0
8,09/30/2021,Joint,KEYS,Stock,Purchase,"$1,001 - $15,000",Thomas H Tuberville,0,0
9,09/30/2021,Joint,KLAC,Stock,Purchase,"$1,001 - $15,000",Thomas H Tuberville,0,0


In [20]:
count = 0

for index,row in new_data.iterrows():
    count = count +1
    if(count == 20):
        break
    ticker = row['ticker']#Ticker

    t_info = row['transaction_date'].split("/") # {09,30,2021}
    t_year = int(t_info[2])
    t_month = int(t_info[0])
    t_day = int(t_info[1])

    start_date = datetime.datetime(t_year,t_month,t_day)
    end_date = start_date + datetime.timedelta(days=1)#next day of start_date
    #end_date = str(int(datetime.datetime(t_year,t_month,t_day+1).timestamp())) 

    #cast vars
    start_date = str(int(start_date.timestamp()))
    end_date = str(int(end_date.timestamp()))

    interval= '1d'
    #'1wk'
    #'1mo'
    #'1yr'
    event ='history'

    url = 'https://query1.finance.yahoo.com/v7/finance/download/'      + ticker + '?period1=' + start_date + '&period2=' + end_date + '&interval='      + interval + '&events=' + event
    spy_url = 'https://query1.finance.yahoo.com/v7/finance/download/'      + "SPY" + '?period1=' + start_date + '&period2=' + end_date + '&interval='      + interval + '&events=' + event
    try:
        data = pd.read_csv(url)
        spy_data = pd.read_csv(spy_url)
    except:
        continue

    basis = (data["Open"][0] + data["Close"][0] + data["High"][0] + data["Low"][0]) / 4
    spy_basis = (spy_data["Open"][0] + spy_data["Close"][0] + spy_data["High"][0] + spy_data["Low"][0]) / 4
    #row['basis'] = basis
    new_data['basis'][index] = basis
    new_data['SPY basis'][index] = spy_basis
    #new_data.set_value(index,'basis',basis)
new_data.head(20)

<ipython-input-20-86216b914160>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['basis'][index] = basis
<ipython-input-20-86216b914160>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['SPY basis'][index] = spy_basis


,transaction_date,owner,ticker,asset_type,type,amount,senator,basis,SPY basis
5,09/30/2021,Joint,PG,Stock,Purchase,"$1,001 - $15,000",Thomas H Tuberville,141,432
6,09/30/2021,Joint,NFLX,Stock,Purchase,"$1,001 - $15,000",Thomas H Tuberville,611,432
7,09/30/2021,Joint,MPWR,Stock,Purchase,"$1,001 - $15,000",Thomas H Tuberville,488,432
8,09/30/2021,Joint,KEYS,Stock,Purchase,"$1,001 - $15,000",Thomas H Tuberville,166,432
9,09/30/2021,Joint,KLAC,Stock,Purchase,"$1,001 - $15,000",Thomas H Tuberville,337,432
10,09/30/2021,Joint,JNJ,Stock,Purchase,"$1,001 - $15,000",Thomas H Tuberville,163,432
11,09/30/2021,Joint,INTU,Stock,Purchase,"$1,001 - $15,000",Thomas H Tuberville,545,432
12,09/30/2021,Joint,GNRC,Stock,Purchase,"$1,001 - $15,000",Thomas H Tuberville,410,432
13,09/30/2021,Joint,EL,Stock,Purchase,"$1,001 - $15,000",Thomas H Tuberville,304,432
14,09/30/2021,Joint,EXP,Stock,Purchase,"$1,001 - $15,000",Thomas H Tuberville,133,432
